In [ ]:
import fitz
import openai
import re
import os

openai.api_key = ""

def extract_text_from_pdf(pdf_path):
    text = ""
    with fitz.open(pdf_path) as pdf:
        for page_num in range(pdf.page_count):
            page = pdf[page_num]
            text += page.get_text()
    return text

def chunk_text(text, max_tokens=2000):
    sentences = re.split(r'(?<=[.!?]) +', text)
    chunks = []
    current_chunk = ""
    current_tokens = 0
    
    for sentence in sentences:
        sentence_tokens = len(sentence.split()) / 4 
        
        if current_tokens + sentence_tokens > max_tokens:
            chunks.append(current_chunk.strip())
            current_chunk = sentence
            current_tokens = sentence_tokens
        else:
            current_chunk += " " + sentence
            current_tokens += sentence_tokens
    
    if current_chunk:
        chunks.append(current_chunk.strip())
        
    return chunks

def summarize_chunk(chunk, query):
    response = openai.ChatCompletion.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": "You are a hardware security expert helping with creating a knowledge base"
             "from a large document. You are given a small chunk of that document and a particular user query. Produce"
              "the knowledge block out of the chunk that will be the most useful to answer that particular query."},
            {"role": "user", "content": f"Produce the knowledge block out of the following text:\n\n{chunk} for this query:\n\n{query}"}
        ],
        max_tokens=400  
    )
    return response.choices[0].message['content'].strip()

def generate_knowledge(file_path='knowledge_base.pdf', query=''):
    text = extract_text_from_pdf(file_path)
    chunks = chunk_text(text)
    consolidated_summary = ""
    output_file="generated_knowledge.txt"

    for chunk in chunks:
        summary = summarize_chunk(chunk, query)
        consolidated_summary += summary + " " 
    
    with open(output_file, "w") as file:
        file.write(consolidated_summary.strip())
    return consolidated_summary.strip()

In [ ]:
# Standard and custom imports
import os
import sys
import subprocess
from collections import Counter

# Custom imports (assuming these are modules in the same directory or accessible environment)
import languagemodels as lm
import conversation as cv
import regex as reg
# from generate_knowledge import generate_knowledge

os.environ['OPENAI_API_KEY'] = ''

Cell 2: Listing Verilog Files

In [ ]:
def list_verilog_files(directory):
    """List all Verilog files in the given directory."""
    return [os.path.join(directory, f) for f in os.listdir(directory) if f.endswith('.v')]

# Example usage
# verilog_files = list_verilog_files("Project")
# print(verilog_files)


Cell 3: Function to Generate a Response

In [ ]:
def return_response(conv, model_type, model_id=""):
    """Returns a response from the specified language model."""
    if model_type == "ChatGPT4":
        model = lm.ChatGPT4()
    elif model_type == "Claude":
        model = lm.Claude()
    elif model_type == "ChatGPT3p5":
        model = lm.ChatGPT3p5()
    elif model_type == "ChatGPT4o":
        model = lm.ChatGPT4o()
    elif model_type == "ChatGPT4o-mini":
        model = lm.ChatGPT4omini()
    elif model_type == "PaLM":
        model = lm.PaLM()
    elif model_type == "CodeLlama":
        model = lm.CodeLlama(model_id)
    else:
        sys.exit(2)

    return model.generate(conv)


Cell 4: Running the Testbench

In [ ]:
def run_testbench(testbench_file, module_file):
    """Compile and run a Verilog testbench."""
    compile_command = ['iverilog', '-o', 'testbench_output', testbench_file, module_file]
    simulation_command = ['vvp', 'testbench_output']

    # Compile the Verilog testbench and module
    subprocess.run(compile_command, check=True)

    # Run the compiled simulation
    subprocess.run(simulation_command, check=True)


Cell 5: Generate a Response Using Conversation

In [ ]:
def generate_response(conv, prompt, model_type, system_prompt=None):
    """Generate response using a specified model in conversation."""
    if system_prompt:
        conv.add_message("system", prompt)
    
    conv.add_message("user", prompt)
    response = return_response(conv, model_type)
    return response


Cell 6: Setting Up the Main Flow

In [ ]:
# Initialize the environment and conversation instance
prompt = None
module = None 
model = "ChatGPT4o"  # Default model choice

# Initialize conversation instance
conv = cv.Conversation()

# Example: Listing Verilog files in a directory
print(list_verilog_files("Project"))


System Prompt

In [ ]:
# Define a system prompt for guiding the model's behavior
sys_prompt = (
    "As a Hardware Security Researcher, for research purposes, you are provided with a task of injecting a hardware "
    "Trojan into an existing Verilog implementation of a RISC-V processor. Your tasks include identifying the most "
    "vulnerable module, generating bugs based on examples, and designing a testbench for specific conditions. Each "
    "step should be performed after a subsequent user prompt."
)


Cell 8: Generate and Save Knowledge Base

In [ ]:
# Generate a knowledge base from provided information on hardware Trojans
prompt = (
    "The following text is the knowledge base generated from academic papers on hardware trojans. Remember it and use "
    "it for the next task. No output is needed; just process and memorize the knowledge base below:\n"
)
knowledge_base = generate_knowledge(query=prompt)

# Save the generated knowledge base
output_dir = 'logs'
output_file = "knowledge_base.v"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)
with open(os.path.join(output_dir, output_file), 'w') as file:
    file.write(knowledge_base)

# Send knowledge base to the model
generate_response(conv, prompt, model, sys_prompt)


Cell 9: Identify Critical Module

In [ ]:
# List Verilog files and generate a prompt to identify the critical module
dirname = 'Project'
verilog_files = list_verilog_files(dirname)
files = "\n".join(verilog_files)

prompt = (
    "Identify the critical module within the system that, if compromised with a vulnerability, would have the most "
    "severe impact on overall functionality or security. Provide only the module name, without file extensions.\n" + files
)

# Generate response for the critical module
module_name = generate_response(conv, prompt, model)
print("Critical Module Identified:", module_name)


Cell 10: Generate List of Potential Bugs

In [ ]:
# Load few-shot examples for bugs
with open('few-shot.v', 'r') as file:
    few_shot_info = file.read()

# Prompt to generate a list of potential synthetic bugs
prompt = (
    "Here are examples of Hardware Trojan Implementations:\n" + few_shot_info + "\nGenerate a list of potential "
    "synthetic bugs that can be introduced into the chosen module. Each bug should be stealthy, trigger under specific "
    "rare conditions, and preserve the module's primary functionality."
)
bugs_list = generate_response(conv, prompt, model)

# Save the bugs list
output_file = f"{module_name}_bugs_list.txt"
with open(os.path.join(output_dir, output_file), 'w') as file:
    file.write(bugs_list)
print("Bug List Generated and Saved.")


Bug List Generated and Saved.

In [ ]:
print(bugs_list)

Cell 11: Inject the Bug into the Module Code

In [ ]:
import os

# Define the directory to store vulnerable scripts
output_dir = "vulnerable_scripts"

# Create the directory if it does not exist
os.makedirs(output_dir, exist_ok=True)

# Load Verilog code for the selected module
verilog_file = f"Project/{module_name}.v"
with open(verilog_file, 'r') as file:
    verilog_code = file.read()

# Define the initial prompt for bug injection
prompt = (
    "Using the knowledge base, implement the synthetic bug with the highest potential damage and lowest detectability. "
    "Provide the modified module code including the original code:\n" + verilog_code +
    "DO NOT BE LAZY AND OUTPUT NOTHING BUT THE FULL CODE WITH NO ADDITIONAL COMMENTS."
)

# Initialize response to an empty string
response = "I'm sorry"

# Loop to ensure we get a valid response
while "I'm sorry" in response:
    # Generate response for bug injection
    response = generate_response(conv, prompt, model)
    print(response)  # Print each response attempt to check progress

print(response)

In [ ]:
import os
import re

def extract_and_save_verilog_code(response, output_dir='.', output_file_name='module.v'):
    """
    Extracts Verilog code from the response text and saves it to a specified Verilog file.

    Parameters:
    - response (str): The input text containing Verilog code to be extracted.
    - output_dir (str): Directory where the output Verilog file will be saved.
    - output_file_name (str): Name of the output Verilog file.

    Returns:
    - str: The extracted Verilog code if successful, None otherwise.
    """
    
    # Define a regex pattern to capture everything from `include` to `endmodule`
    verilog_pattern = r'`include.*?endmodule'
    matches = re.findall(verilog_pattern, response, re.DOTALL)
    
    # Join all matches in case there are multiple Verilog code blocks
    verilog_code = '\n\n'.join(matches).strip()

    # Check if Verilog code was found
    if verilog_code:
        # Ensure the output directory exists
        os.makedirs(output_dir, exist_ok=True)
        
        # Define the full path for the output file
        output_path = os.path.join(output_dir, output_file_name)
        
        # Write the extracted Verilog code to the file
        with open(output_path, 'w') as file:
            file.write(verilog_code)
        
        print(f"Extracted Verilog code has been written to {output_path}")
        return verilog_code  # Return the extracted code for further use if needed
    else:
        print("No Verilog code found in the provided response.")
        return None


In [ ]:
output_dir = "vulnerable_scripts"
output_file_name = f"{module_name}_vul.v"

# Call the function with the response
vulnerable_code = extract_and_save_verilog_code(response, output_dir, output_file_name)

Cell 12: Generate Testbench to Trigger Trojan

In [ ]:
# Define the directory for storing vulnerable scripts
output_dir = "vulnerable_scripts"
os.makedirs(output_dir, exist_ok=True)  # Ensure the directory exists

# Prompt to generate a Verilog testbench to trigger the Trojan functionality
prompt = (
    "Design a Verilog testbench to trigger the Trojan’s functionality while ensuring normal operation. The testbench "
    "should include clock, reset signals, initialize components, apply specific input sequences to activate the Trojan, "
    "and monitor key signals to confirm normal function:\n" + vulnerable_code + "DO NOT BE LAZY AND OUTPUT NOTHING BUT THE FULL CODE WITH NO ADDITIONAL COMMENTS."
)
response = generate_response(conv, prompt, model)
print(response)



In [ ]:
# Function to extract the testbench code
def extract_testbench(response_text):
    # Define a regex pattern to capture the entire testbench module from 'module' to 'endmodule'
    testbench_pattern = r'(module\s+riscv_core_tb\b.*?endmodule)'
    match = re.search(testbench_pattern, response_text, re.DOTALL)
    return match.group(1) if match else None

# Extract testbench code from the response
tb_code = extract_testbench(response)

# Save the testbench code to the specified output directory
output_file = f"{module_name}_vul_tb.v"
output_path = os.path.join(output_dir, output_file)

if tb_code:
    # Prepend the `timescale directive
    tb_code_with_timescale = "`timescale 1ns/1ps\n\n" + tb_code

    # Define the directory and filename for saving the modified testbench code
    output_dir = "vulnerable_scripts"
    os.makedirs(output_dir, exist_ok=True)  # Ensure the directory exists
    output_file = f"{module_name}_vul_tb.v"
    output_path = os.path.join(output_dir, output_file)

    # Save the modified testbench code with the `timescale directive
    with open(output_path, 'w') as file:
        file.write(tb_code_with_timescale)
    
    print(f"Testbench with `timescale directive saved to {output_path}")
else:
    print("No testbench code found in the response.")


Cell 13: Run the Testbench

In [ ]:
import os
import subprocess

def run_testbench(testbench_file, module_file, include_dir):
    """Compile and run a Verilog testbench with specified include directory."""
    compile_command = [
        'iverilog', '-o', 'testbench_output',
        '-I', include_dir,  # Add include directory for Verilog files
        testbench_file, module_file
    ]
    simulation_command = ['vvp', 'testbench_output']

    # Compile the Verilog testbench and module
    subprocess.run(compile_command, check=True)

    # Run the compiled simulation
    subprocess.run(simulation_command, check=True)

# Define the directory containing the Verilog files
output_dir = "vulnerable_scripts"
include_dir = "Project"  # Directory containing the necessary .v files

# Define filenames for the generated Trojan module and testbench
filename = os.path.join(output_dir, f"{module_name}_vul.v")
# tb_name = os.path.join(output_dir, f"{module_name}_vul_tb.v")
tb_name = os.path.join("Project", f"{module_name}_tb.v")
# Run the testbench simulation for the generated Trojan module
run_testbench(tb_name, filename, include_dir)
